In [2]:
from keras_synthetic_genome_sequence import GapSequence
from keras_synthetic_genome_sequence.utils import get_gaps_statistics
from ucsc_genomes_downloader import Genome
from ucsc_genomes_downloader.utils import tessellate_bed
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon, pearsonr, ks_2samp, chisquare

In [13]:
np.float

float

In [10]:
from humanize import naturalsize

In [3]:
h = Genome("hg19").filled()

In [12]:
naturalsize((h.chromEnd-h.chromStart).sum()*4*8)

'91.5 GB'

In [9]:
assemblies = ("hg19", "hg38", "mm9", "mm10")

In [10]:
window_size = 200
batch_size = 50000
max_gap_size = 3

In [12]:
for assembly in assemblies[1:2]:
    number, mean, covariance = get_gaps_statistics(
        assembly=assembly,
        max_gap_size=max_gap_size,
        window_size=window_size
    )

    genome = Genome(assembly, chromosomes=["chr1"])
    ground_truth = tessellate_bed(genome.filled(), window_size=window_size)

    gap_sequence = GapSequence(
        assembly=assembly,
        bed=ground_truth,
        gaps_mean=mean,
        gaps_covariance=covariance,
        batch_size=batch_size
    )
    
    X, y = gap_sequence[0]
    synthetic_mean = np.isclose(X, 0.25).all(axis=-1).mean(axis=0)
    
    plt.bar(range(len(synthetic_mean)), synthetic_mean, width=1)
    plt.title(f"Synthetic gaps frequencies ({assembly})")
    plt.ylabel("Synthetic gaps frequency")
    plt.xlabel("Nucleotides position")
    plt.savefig(f"synthetic_gaps_{assembly}.png")
    plt.close()
    
    plt.bar(range(len(mean)), mean, width=1)
    plt.title(f"Biological gaps frequencies ({assembly})")
    plt.ylabel("Biological gaps frequency")
    plt.xlabel("Nucleotides position")
    plt.savefig(f"biological_gaps_{assembly}.png")
    plt.close()
    
    print(assembly)
    print("Gaps number", number)
    for test in (wilcoxon, ks_2samp):
        print(test.__name__, test(mean, synthetic_mean))

hg38
Gaps number 235
wilcoxon WilcoxonResult(statistic=7419.0, pvalue=0.0025966013781234085)
ks_2samp Ks_2sampResult(statistic=0.24, pvalue=1.8266119303942767e-05)
